If you don't have Qiskit installed, you'll want to go ahead and run:

In [ ]:
pip install qiskit --upgrade

In [ ]:
pip install numpy --upgrade

And now to make sure everything is installed correctly:

In [ ]:
import qiskit.tools.jupyter
%qiskit_version_table
%qiskit_copyright

First, we'll load in the sequences we would like to compare.

In [ ]:
a = "AACCAGTCG"
b = "CACCGTAA"

Next, we need to generate a substitution matrix of matches and non-matches using the classical Smith-Waterman algorithm.

In [ ]:
import itertools
import numpy as np

# Initialize an (n+1)*(m+1) matrix
# Where n and m are len(a) and len(b) respectively
H = np.zeros((len(a) + 1, len(b) + 1), np.int)

# Compare every character in a against every character in b
# Matches (white cells) are denoted with 1
for i, j in itertools.product(range(1, H.shape[0]), range(1, H.shape[1])):
  if a[i - 1] == b[j - 1]:
    H[i, j] = 1

# print("Before removing extra rows:")
# print(H)

# TODO
# Verify this doesn't break if we swap a and b
for x in range(0, abs(H.shape[0] - H.shape[1])):
  H = np.delete(H, 0, 0)
  H = np.delete(H, 0, 1)

# print("After removing extra rows:")
print(H)

# print(H.shape[0])
# print(H.shape[1])

We now need to transform the matrix H such that the diagonals become horizontal or vertical lines. This is necessary since we do not have access to a spatial light modulator.

In [ ]:
# I'm really unsure if this is the right approach

diag_H = np.array([])

diagonal = np.diag(H, k=0)
# print(diagonal)

diag_len = len(diagonal)
diag_H = diagonal

for i in range(1, H.shape[1]):
  diagonal = np.diag(H, k=i)
  # print(diagonal)

  # If the diagonal is shorter than the rows, append zeros
  if diag_len > len(diagonal):
    zeros = np.zeros((1, (diag_len-len(diagonal))), dtype=int)
    diagonal = np.append(diagonal, zeros)
    diag_H = np.vstack((diag_H, diagonal))

  # Otherwise append the diagonal as a row
  else:
    diag_H = np.vstack((diag_H, diagonal))

print(diag_H)

Encode the array such that it can become the input to the quantum algorithm

In [ ]:
# TODO

Next, we need to apply Schützhold's Quantum Algorithm.

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit.library import QFT

# Create a 3 qubit quantum register
# qr[0] is x, qr[1] is y, qr[2] is f(x,y)
qr = QuantumRegister(3)
cr = ClassicalRegister(3)

circuit = QuantumCircuit(qr, cr)

%matplotlib inline

circuit.h(qr[0])
circuit.h(qr[1])

# Feed the circuit into the BB
# TODO
sub_q = QuantumRegister(3)
sub_circ = QuantumCircuit(sub_q, name='BB')
sub_inst = sub_circ.to_instruction()
circuit.append(sub_inst, [qr[0], qr[1], qr[2]])

# QFT on qr[0] and qr[1]
qft = QFT(num_qubits=2)
circuit.compose(qft, inplace=True)

# Measure all 3
circuit.measure(qr, cr)

circuit.draw(output='mpl')